In [ ]:
!pip install tqdm pandas langchain -i https://pypi.douban.com/simple/ 

In [ ]:
import requests
import json
import time
import pandas as pd
from IPython.display import Markdown  
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
api_keys=['sk-1',"sk-2","sk-3","sk-4"]
api_keys

In [ ]:
class APIPool:
    def __init__(self,tokens):
        self.tokens=tokens
        self.last_called={token:0 for token in tokens}  #上次调用时间
    def getapi(self):
        now = int(time.time())
        for token in self.tokens:
            if self.last_called[token]+21 <=now:    
                self.last_called[token]=now
                return token
        time.sleep(self.last_called[self.tokens[0]]+21-now) #等待
        return self.getapi() #重新调用
    def reset(self):
        self.last_called={token:0 for token in self.tokens}#重置

In [ ]:
url = "https://api.lixining.com/v1/chat/completions" # 选择一个API
#反向代理 如失效需要 更换到上面
#https://openai.1rmb.tk/v1/chat/completions
#https://ai.fakeopen.com/v1/chat/completions
#https://api.lixining.com/v1/chat/completions
A=APIPool(api_keys) #创建API池

In [ ]:
def get_header():# 获取请求头
    header={
  'Authorization': f'Bearer {A.getapi()}',
  'Content-Type': 'application/json'
    }
    return header

In [ ]:
#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "system", "content": f"You are ChatGPT, a large language model trained by OpenAI. \
            Follow the user's instructions carefully. Respond using markdown."},
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    data=None
    try:
        response = requests.post(url, headers=get_header(), json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

In [ ]:
get_completion("hello world")

In [ ]:
review_1 = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""
# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [ ]:
len(review_1)+len(review_2)+len(review_3)+len(review_4)

In [ ]:
def user(text):
    prompt = f"""
    Your task is to extract relevant information from \ 
    a product review from an ecommerce site to give \
    feedback to the Shipping department. 

    From the review below, delimited by triple quotes \
    extract the information relevant to shipping and \ 
    delivery. Limit to 30 words. 

    Review: ```{prod_review}```
    """

    response = get_completion(prompt)
    return (response)

In [ ]:
for i in reviews:
    user(i)

In [ ]:
class TextStorage:
    def __init__(self):
        self.text = ""

    def add_text(self, new_text):
        self.text += str(new_text)

    def get_text(self):
        return self.text
    
    def clear_text(self):
        self.text = ""

In [ ]:
ltext=f"""
道德经简体全文版
道德经·第一章：观庙
道可道，非常道。名可名，非常名。无名天地之始。有名万物之母。故常无欲以观其妙。常有欲以观
其徼。此两者同出而异名，同谓之玄。玄之又玄，众妙之门。
道德经·第二章：观徼
天下皆知美之为美，斯恶矣；皆知善之为善，斯不善已。故有无相生，难易相成，长短相形，高下相
倾，音声相和，前後相随。是以圣人处无为之事，行不言之教。万物作焉而不辞。生而不有，为而不恃，
功成而弗居。夫唯弗居，是以不去。
道德经·第三章：安民
不尚贤，使民不争。不贵难得之货，使民不为盗。不见可欲，使民心不乱。是以圣人之治，虚其心，
实其腹，弱其志，强其骨；常使民无知、无欲，使夫智者不敢为也。为无为，则无不治。
道德经·第四章：不盈
道冲而用之，或不盈。渊兮似万物之宗。解其纷，和其光，同其尘，湛兮似或存。吾不知谁之子，象
帝之先。
道德经·第五章：守中
天地不仁，以万物为刍狗。圣人不仁，以百姓为刍狗。天地之间，其犹橐迭乎？虚而不屈，动而愈出
。多言数穷，不如守中。
道德经·第六章：谷神
谷神不死是谓玄牝。玄牝之门是谓天地根。绵绵若存，用之不勤。
道德经·第七章：无私
天长地久。天地所以能长且久者，以其不自生，故能长生。是以圣人後其身而身先，外其身而身存。
非以其无私邪！故能成其私。
道德经·第八章：若水
上善若水。水善利万物而不争，处众人之所恶，故几於道。居善地，心善渊，与善仁，言善信，正善
治，事善能，动善时。夫唯不争，故无尤。
道德经·第九章：持盈
持而盈之不如其己；揣而锐之不可长保；金玉满堂莫之能守；富贵而骄，自遗其咎。功遂身退，天之
道。
道德经·第十章：玄德
载营魄抱一，能无离乎？专气致柔，能如婴儿乎？涤除玄览，能无疵乎？爱国治民，能无为乎？天门
开阖，能为雌乎？明白四达，能无知乎。
道德经·第十一章：虚中
三十幅共一毂，当其无，有车之用。埏埴以为器，当其无，有器之用。凿户牖以为室，当其无，有室
之用。故有之以为利，无之以为用。
道德经·第十二章：为腹
五色令人目盲，五音令人耳聋，五味令人口爽，驰骋畋猎令人心发狂，难得之货令人行妨。是以圣人
，为腹不为目，故去彼取此。
道德经·第十三章：宠辱
宠辱若惊，贵大患若身。何谓宠辱若惊？宠为下。得之若惊失之若惊是谓宠辱若惊。何谓贵大患若身
？吾所以有大患者，为吾有身，及吾无身，吾有何患。故贵以身为天下，若可寄天下。爱以身为天下，若
可托天下。
道德经·第十四章：道纪
视之不见名曰夷。听之不闻名曰希。抟之不得名曰微。此三者不可致诘，故混而为一。其上不皦
（JIǍo），其下不昧，绳绳不可名，复归於无物。是谓无状之状，无物之象，是谓惚恍。迎之不见其首，随
之不见其後。执古之道以御今之有。能知古始，是谓道纪。
道德经·第十五章：不盈
古之善为士者，微妙玄通，深不可识。夫唯不可识，故强为之容。豫兮若冬涉川；犹兮若畏四邻；俨
兮其若容；涣兮若冰之将释；敦兮其若朴；旷兮其若谷；混兮其若浊；澹兮其若海；飉（Liáo，风的声音）兮
若无止。孰能浊以静之徐清。孰能安以动之徐生。保此道者不欲盈。夫唯不盈故能蔽而新成。
道德经·第十六章：复命
致虚极守静笃。万物并作，吾以观复。夫物芸芸各复归其根。归根曰静，是谓复命；复命曰常，知常
曰明。不知常，妄作凶。知常容，容乃公，公乃全，全乃天，天乃道，道乃久，没身不殆。
道德经·第十七章：知有
太上，下知有之。其次，亲而誉之。其次，畏之。其次，侮之。信不足焉，有不信焉。悠兮其贵言，
功成事遂，百姓皆谓：我自然。
道德经·第十八章：四有
大道废有仁义；慧智出有大伪；六亲不和有孝慈；国家昏乱有忠臣。
道德经·第十九章：朴素
绝圣弃智，民利百倍；绝仁弃义，民复孝慈；绝巧弃利，盗贼无有；此三者，以为文不足。故令有所
属，见素抱朴少私寡欲。
道德经·第二十章：食母
绝学无忧，唯之与阿，相去几何？善之与恶，相去若何？人之所畏，不可不畏。荒兮其未央哉！众人
熙熙如享太牢、如春登台。我独泊兮其未兆，如婴儿之未孩；儡儡（Lěi，羸弱）兮若无所归。众人皆有馀，而
我独若遗。我愚人之心也哉！沌沌兮。俗人昭昭，我独昏昏；俗人察察，我独闷闷。众人皆有以，而我独
顽且鄙。我独异於人，而贵食母。
"""

In [ ]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """
    response = get_completion(prompt)
    print(i, response, "\n")

In [ ]:
len(ltext)

In [ ]:
text_spl= RecursiveCharacterTextSplitter(separators=["\n\n","\n"],chunk_size=450)

In [ ]:
docs=text_spl.create_documents([ltext])

In [ ]:
docs

In [ ]:
print(f"现在已经分割{len(docs)}个部分，准备喂到模板里")

In [ ]:
T=TextStorage()

In [ ]:
def chatgpt(text):
    prompt=f"""我要你担任道教哲学老师，已经研究道德经的专家。下面三个引号括起来的文本是道德经的原文，你的工作就是用通俗易懂的方式解释这些概念。\
        这可能包括提供示例、提出问题或将复杂的想法分解成更容易理解的更小的部分,用markdown格式回复我。
    \"\"\"{text}\"\"\"
    """
    response = get_completion(prompt)
    return (response)

In [ ]:
T.clear_text()

In [ ]:
for d in tqdm(docs):
    C_text=chatgpt(d.page_content)
    print("-"*10)
    T.add_text(C_text)
print("写入完成")

In [ ]:
Markdown(T.get_text()) 